# Ingredient Standardization

## Overview

The purpose of the exercise is to identify similar ingredients and replace them with one main ingredient. This exercise will reduce the dimensionality of an ingredients list, which can then be piped into other methods for processing. 

The main task is to identify the **semantic similarity** of ingredients. Semantic similarity falls under the more general concept of semantic relatedness. Semantic relatedness refers to measuring any relationship between two words (such as "rain" and "flood"). Semantic similarity more specifically measures the interchangeability of words (such as "bank" and "trust company"). I will work with semantic similarity instead of semantic relatedness for this analysis because I want to determine which ingredients are essentially interchangeable, as opposed to ingredients that are parts of, or related to, other ingredients.

Within text analysis exist, two main methods to determine semantic similarity. The first method is **corpus-based semantic similarity**. Corpus-based (or "information content") methods measure the differences in information content between two words or phrases as a function of their probability of occurrence in a corpus. The second method, **edge counting semantic similarity**, defines text similarity as a function of the length of a path linking two words or phrases together and on the position of the word or phrases in a taxonomy. Edge counting methods typically use hierarchical word networks, such as WordNet, to determine path similarities.

This exercise is challenging because of many factors. Ingredients are generally not single words, but are word phrases. Additionally, a great amount of variation exists within a particular ingredient type. For example, [give example]. 

In this analysis, I will attempt both methods of semantic similarity. First, I will derive cosine similarity of every ingredient with every other ingredient in the ingredient list. For each ingredient, I extract the top ten similarities. Second, I will attempt to cull the list of similarities obtained by the cosine similarity exercise using a path similarities computational algorithm. I believe computing path similarities for each ingredient paring in the list would be too computationally intensive.

Both semantic similarity methods are essentially grouping ingredients togther. Measures of semantic similarity are not straightforward to validate. As an alternative method of developing and validating groupings, my third step will be to perform a k-means clustering. 


## Importing packages and loading data

In [2]:
import pandas as pd
from itertools import chain
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import re
import pickle
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
import matplotlib.pyplot as plt

In [3]:
# Load JSON ingredient data
recipes = pd.read_json("recipes.json")

In [4]:
# Load validation set
validationSet = pickle.load(open("validationSet.py", "rb"))

## Cleaning the text and prepping it for analysis

In [5]:
# Pull ingredients into a list
ingredients = list(chain.from_iterable(recipes.ingredients.tolist()))
ingredientsLower = [i.lower() for i in ingredients]

In [6]:
# Extract unique ingredients
ingredientsLower = list(set(ingredients))
len(ingredientsLower)

6714

In [7]:
# Remove the validation (test) ingredients from the main dataset
ingredientsLower_train = [i for i in ingredientsLower if i not in validationSet] 
len(ingredientsLower_train)

6559

In [8]:
# Identify words in the rightmost stage of the phrase (the "anchor" words)
last_tokens = [i.lower().split()[-1] for i in ingredientsLower_train]
last_tokens = set(last_tokens)

In [9]:
# Stem words and take the unique entries
ps = PorterStemmer()
last_tokens = set([ps.stem(w) for w in last_tokens])

## TF-IDF Vectorization (Word Analyzer) and Cosine Similarity

The first step in any text analysis is to vectorize words. I vectorize the ingredients using the TF-IDF algorithm. TF-IDF is a commonly used word vectorizer. In the TF-IDF vectorizer call, the "text analyzer" we will use is "word;" essentially, I am using the word as the elements that populate the vectors. Alternative analyzers are at the character- and n-gram-level. 

In [10]:
# Obtain a TF-IDF matrix of vectors to which we can apply the cosine similarity algorithm
vectorizer = TfidfVectorizer(min_df =1, analyzer = "word")
tfidf = vectorizer.fit_transform(ingredientsLower_train)

According to https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity, the linear kernel dot product is functionally the same as the cosine similarity, because the TF-IDF vectors are normalized. 

In [11]:
# Get the cosine similarity of the first ingredient ("document") with every other ingredient    
cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()

In [12]:
# What are the top five matches?
indicies = cosine_similarities.argsort()[:-5:-1]
indicies

array([   0, 5412, 2995, 5509])

Now, I'll loop over all entries in the tfidf matrix and pull out the top ten similarities. I am finding similarities this way because computing the TF-IDF matrix of every document with every other document gave me nonsense indicies that did not correspond to the ingredients dataset.

In [13]:
similarities = []

for i in range(0, len(ingredientsLower_train) - 1 ):
    cosine_similarities = linear_kernel(tfidf[i], tfidf).flatten()
    indicies = cosine_similarities.argsort()[:-10:-1]
    similarities.append({ingredientsLower_train[i] : [ingredientsLower_train[i] for i in indicies]})

del i, vectorizer, tfidf, cosine_similarities, indicies

The next step is to validate the effectiveness of this method. I'll create a data frame where each row is a match between ingredients. I need a way to determine whether these methods are valid. From inspection, it seems like each ingredient phrase has an "anchor" ingredient. I will judge matches as good, then, if they share the same anchor ingredient and if the TF-IDF algorithm placed them in the top ten. This may not be the best measure of positive matches, but it's the easiest to derive a confusion matrix from.

In [14]:
# Pull the ingredient list into a data frame of pairwise combinations
ingredientMatches = pd.DataFrame(columns = ['key', 'match'])
for match in similarities:
    ingredientMatches = ingredientMatches.append({'key': "".join(match.keys()), 'match': [i for i in match.values()][0]}, ignore_index = True)
ingredientMatches.head()

,key,match
0,pork tongue,"[pork tongue, tongue, beef tongue, veal tongue..."
1,chicken stock cubes,"[chicken stock cubes, Knorr Chicken Stock Cube..."
2,boneless skinless chicken,"[boneless skinless chicken, boneless, skinless..."
3,poolish,"[poolish, nut oil, cayenne pepper sauce, sprin..."
4,banh hoi,"[banh hoi, banh pho rice noodles, nut oil, bla..."


In [15]:
# Unnest the list to obtain pairwise matches and drop duplicates
ingredientMatches = ingredientMatches.set_index("key").apply(lambda x: x.apply(pd.Series).stack()).reset_index(level=1, drop=True).reset_index()
ingredientMatches.head()

,key,match
0,pork tongue,pork tongue
1,pork tongue,tongue
2,pork tongue,beef tongue
3,pork tongue,veal tongue
4,pork tongue,pork
5,pork tongue,ox tongue
6,pork tongue,ground pork
7,pork tongue,salt pork
8,pork tongue,pork stock
9,chicken stock cubes,chicken stock cubes


In [16]:
ingredientMatches = ingredientMatches.drop_duplicates()
ingredientMatches.shape

(59022, 2)

In [17]:
# Define a new column which identifies whether a match is valid
ingredientMatches['true_pos'] = ingredientMatches['key'].str.split().apply(lambda x: x[-1]) == ingredientMatches['match'].str.split().apply(lambda x: x[-1])
ingredientMatches['false_pos'] = ingredientMatches['key'].str.split().apply(lambda x: x[-1]) != ingredientMatches['match'].str.split().apply(lambda x: x[-1])
ingredientMatches['tag'] = 1
ingredientMatches.head()

,key,match,true_pos,false_pos,tag
0,pork tongue,pork tongue,True,False,1
1,pork tongue,tongue,True,False,1
2,pork tongue,beef tongue,True,False,1
3,pork tongue,veal tongue,True,False,1
4,pork tongue,pork,False,True,1


In [18]:
# Append the rest of the ingredient-matches to the ingredient matches list
keylist = ingredientMatches.drop_duplicates(subset = "key").key.tolist()
temp = list(ingredientMatches[['key', 'match']].itertuples(index = False, name = None))

for item in keylist:
    for i in ingredientsLower_train:
        if (item, i) not in temp:
            ingredientMatches = ingredientMatches.append({"key": item, "match": i, "true_pos": False, "false_pos": False, "tag": 0}, ignore_index = True)


KeyboardInterrupt: 

In [20]:
# Calculate true and false negatives
ingredientMatches['true_neg'] = ingredientMatches['key'].str.split().apply(lambda x: x[-1]) != ingredientMatches['match'].str.split().apply(lambda x: x[-1]) and ingredientMatches['tag'] ==0
ingredientMatches['false_neg'] = ingredientMatches['key'].str.split().apply(lambda x: x[-1]) == ingredientMatches['match'].str.split().apply(lambda x: x[-1]) and ingredientMatches['tag'] ==0

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
import pickle
pickle.dump(ingredientMatches, "ingredientMatches.py")

Plot the ROC curve

In [ ]:
true_positive_rate = ingredientMatches['true_pos'].sum() /  ingredientMatches['true_pos'].sum() + ingredientMatches['false_neg'].sum()
false_positive_rate = ingredientMatches['false_pos'].sum() /  ingredientMatches['false_pos'].sum() + ingredientMatches['true_neg'].sum()

plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


## TF-IDF Vectorization ("Anchor" Analyzer) and Cosine Similarity

In [ ]:
# Identify ingredients which contain the anchor word
position = []
for i in last_tokens:
    if i in vectorizer.vocabulary_.keys():
        position.append(vectorizer.vocabulary_[i])

In [ ]:
# Place greater weights on entries in the TF-IDF matrix that contain the anchor word. The weight is notional
tfidf[:, position] *= 5.0

In [ ]:
# Obtain word similarities
similarities = []

for i in range(0, len(ingredientsLower_train) - 1 ):
    cosine_similarities = linear_kernel(tfidf[i], tfidf).flatten()
    indicies = cosine_similarities.argsort()[:-5:-1]
    similarities.append({ingredientsLower_train[i] : [ingredientsLower_train[i] for i in indicies]})

del i, vectorizer, tfidf, cosine_similarities, indicies

similarities[0:5]

What I still did not realize when implementing this was that this is essentially just scaling the weights on all the entries up, since all the ingedients have an "anchor" ingredient. One to-do item for me is to find a smarter weighting method that differentially weights anchor ingredients higher if they're already assigned as matches to the main ingredient.  

Where I'm left is with a bunch of matches to a particular ingredient, some of which are relevant and some of which are not. What are ways we can weed out the irrelevant ingredients? One thought is to use Wordnet synsets. NLTK's Wordnet interface has functions to compute path similarities, as well as determine word hyponyms and hypernyms. The first method I will try is to weed out matched ingredients with a path similarity from the anchor word that's lower than some arbitrary threshold. Another method for determining "families" of words is to compare hypernyms of words within a phrase and only keep words under the same family of words. Instructions on using the NLTK implementation of Wordnet can be found here: http://www.nltk.org/howto/wordnet.html.

## Using synsets as a validation method

In [ ]:
# Define a function which determines the synsets of the rightmost word
def find_synset(split):
    for i in range(1, len(split)+1):        
        try:
            synset = wn.synset(split[-i] +'.n.01')
            break
        except:
            pass
    return [synset, anchor]

In [ ]:
# Define a function to further weed out irrelevant entries identified by TF-IDF
newSimilarities = {}
def cull_similarities(textbox):
    global newSimilarities
    for i in range(0, len(textbox)):
        keep = []
        text = "".join(textbox[i].keys()).split()
        key = "".join(textbox[i].keys())

        # Determine rightmost viable ingredient and its synset
        test = ps.stem(text[-1])
        anchor = wn.synset(test +'.n.01')
        print(anchor)
    
        # Compare matches
        matches = [i for i in textbox[i].values()][0]
    
        for i in matches:
            paths = []
            if i.split()[-1] == anchor.lemma_names()[0]:
                keep.append(i)
            else:
                try:
                     for j in i.split():
                         paths.append(anchor.path_similarity(wn.synset(j + '.n.01')))
                except:
                    paths.append(0)
            #print(i + ": " + str(np.mean(paths)))
            if np.mean(paths) >= 0.08:
                keep.append(i)
        # Append matches to a new dictionary
        newSimilarities[key] = keep

In [ ]:
cull_similarities(similarities)

## K-Means Clustering

Text clustering uses a text vector to identify groupings with similar patterns

In [ ]:
# Source: http://brandonrose.org/clustering#K-means-clustering
k = 100 # Generate 100 clusters; this number is arbitrary and reduces the work on me to check the clusters
km = KMeans(n_clusters = k)
km.fit(tfidf)
clusters = km.labels_.tolist()

In [ ]:
# Identify ingredients that characterize each cluster
temp = pd.DataFrame({"ingredients": ingredientsLower_train})

order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(k):
    print("Cluster %d ingredients:" % i, end='')
    
    for ind in order_centroids[i, :5]: 
        print(' %s' % temp.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
